## Measuring a PSF for Spliner Analysis

In this example we'll measure a microscope PSF for Spliner analysis.

### Configuration

To measure the PSF you'll need a movie of sparse fluorescent beads being scanned through the microscope focus. We typically do this with small fluorescent beads on the order of 0.1um in diameter. The beads are fixed to the coverslip using a buffer with 100mM MgCl2. The coverslip is scanned through the focus in 10nm steps using a piezo z scanner. We assume that the drift in XYZ is neglible during the time it takes to take these movies (10s of seconds).

In this example we're just going to simulate this using a theoritical PSF that is an idealized astigmatic PSF. 

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate the sample data for this example.

In [ ]:
import storm_analysis.jupyter_examples.spliner_measure_psf as spliner_measure_psf
spliner_measure_psf.configure()

### Prerequisites

A text file containing the z-offset of each frame in the movie. This file contains two columns, the first is whether or not the data in this frame should be used (0 = No, 1 = Yes) and the second contains the z offset in microns.

In [ ]:
# Load sample z_offsets.txt file. In this example every frame is valid.

import numpy
z_offsets = numpy.loadtxt("z_offsets.txt")

print(z_offsets[:5,:])

A text file containing the approximate bead locations. One way to create this is by using the `visualizer` tool in the `storm_analysis.visualizer` folder. Another alternative might be using ImageJ. Here we are just going to use generated file.

In [ ]:
bead_locs = numpy.loadtxt("bead_locs.txt")
print(bead_locs)


In [ ]:
# Check that the specified positions align with the actual bead locations.
#
# spliner_measure.tif is the name of the simulated movie.
#
import storm_analysis.jupyter_examples.overlay_image as overlay_image
overlay_image.overlayImageBeads("spliner_measure.tif", "bead_locs.txt", 40)

### PSF Measurement

Now that we have the 3 files that we need we can measure the PSF. You want to use an AOI that is a bit larger then the size you plan to use when creating the spline file. 

Notes:

* The AOI size is the 1/2 size so using 12 pixels means that the final PSF will be 24 x 24 pixels.
* The pixel size in this simulation was 100nm
* The simulated movie covered a Z range of +-0.6um, so we're using exactly that. For real data you might want to use a Z range that is slightly smaller than range you scanned.
* The Z step size is 50nm, which is fine enough for most applications, assuming a high NA objective with a depth of field on the order of 1um.

Also: 

If you think that your coverslip might not be completely flat, or that your bead locations are not sufficiently accurate then you can set the `refine` parameter to `True`. With this setting each PSF will be aligned to the average of the remaining PSF using a correlation metric. You may see some warning messages like `Warning: Desired error not necessarily achieved due to precision loss.`, it is usually safe to ignore these.

In [ ]:
import storm_analysis.spliner.measure_psf_beads as measure_psf_beads

measure_psf_beads.measurePSFBeads("spliner_measure.tif", 
                                  "z_offsets.txt",
                                  "bead_locs.txt",
                                  "spliner_psf.psf",
                                  aoi_size = 12,
                                  pixel_size = 0.1,
                                  refine = True,
                                  z_range = 0.6,
                                  z_step = 0.05)
                                  

### Visualizing the measured PSF

Make some images of the measured PSF.

In [ ]:
import storm_analysis.jupyter_examples.psf_images as psfImages

psfImages.psfImages("spliner_psf.psf")

Print some properties of the PSF.

In [ ]:
import storm_analysis.spliner.print_psf as printPSF

printPSF.printPSF("spliner_psf.psf")

Now you can use `storm_analysis.spliner.psf_to_spline` to convert the PSF into a spline that `Spliner` can use for SMLM movie analysis